In [4]:

import pandas as pd
import numpy as np
import seaborn as sns

from sklearn import linear_model
from datetime import datetime as dt

# #Establecemos el directorio
# user = os.environ.get('USERNAME')
# os.chdir('C:'+ '\\' + 'Users' + '\\' + user + '\\' + 'Desktop')

#correr es solo para que muestre los numeros completos (sin notacion cientifica)
pd.options.display.float_format = '{:.2f}'.format

data = pd.read_csv(
    r'C:\Users\andre\Desktop\Scripts\VentasPorProveedor.csv',
    encoding ='latin1',
    engine ="python",
    sep =';',
    on_bad_lines ='skip')

traindata = pd.read_excel(
    r'C:\Users\andre\Desktop\Scripts\DATASETS\XLSX\online_retail_II.xlsx',
    sheet_name = 'Year 2009-2010'
)



df_train = pd.DataFrame(
    traindata)

df_train.head()



,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.00,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.00,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.00,United Kingdom


In [5]:
df_train.describe()

,Quantity,Price,Customer ID
count,525461.00,525461.00,417534.00
mean,10.34,4.69,15360.65
std,107.42,146.13,1680.81
min,-9600.00,-53594.36,12346.00
25%,1.00,1.25,13983.00
50%,3.00,2.10,15311.00
75%,10.00,4.21,16799.00
max,19152.00,25111.09,18287.00


In [6]:
df_train["InvoiceDate"] = df_train["InvoiceDate"].astype(str)

date = df_train["InvoiceDate"].str.split(expand=True)
date.columns = ['FInvoiceDate', 'InvoiceTime']

df_train = pd.concat([df_train, date], axis=1)

df_train.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,FInvoiceDate,InvoiceTime
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.00,United Kingdom,2009-12-01,07:45:00
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom,2009-12-01,07:45:00
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom,2009-12-01,07:45:00
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.00,United Kingdom,2009-12-01,07:45:00
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.00,United Kingdom,2009-12-01,07:45:00


In [7]:
#Now we reduce the original dataframe to a new one with the columns we need
df_train = pd.DataFrame(
    df_train,
    columns= [
    "Invoice",
    "FInvoiceDate",
    "Customer ID",
    "Country",
    "StockCode",
    "Description",
    "Quantity",
    "Price"]
)

# df_train.drop(
#     ["InvoiceDate, InvoiceTime"],
#     axis= 1)

df_train.head()

,Invoice,FInvoiceDate,Customer ID,Country,StockCode,Description,Quantity,Price
0,489434,2009-12-01,13085.00,United Kingdom,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,6.95
1,489434,2009-12-01,13085.00,United Kingdom,79323P,PINK CHERRY LIGHTS,12,6.75
2,489434,2009-12-01,13085.00,United Kingdom,79323W,WHITE CHERRY LIGHTS,12,6.75
3,489434,2009-12-01,13085.00,United Kingdom,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2.10
4,489434,2009-12-01,13085.00,United Kingdom,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.25


In [8]:
df_train["Customer ID"].astype("string")

df_train.head()

,Invoice,FInvoiceDate,Customer ID,Country,StockCode,Description,Quantity,Price
0,489434,2009-12-01,13085.00,United Kingdom,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,6.95
1,489434,2009-12-01,13085.00,United Kingdom,79323P,PINK CHERRY LIGHTS,12,6.75
2,489434,2009-12-01,13085.00,United Kingdom,79323W,WHITE CHERRY LIGHTS,12,6.75
3,489434,2009-12-01,13085.00,United Kingdom,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2.10
4,489434,2009-12-01,13085.00,United Kingdom,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.25


Now we call df_try a new dataframe to validate the model

In [9]:
import matplotlib.pyplot as plt
import numpy as np

df_grouped = df_train.groupby("Country").agg({"Quantity": "mean", "Price": "mean"})

corr_matrix = df_train[["Quantity", "Price"]].corr()

print(corr_matrix)


          Quantity  Price
Quantity      1.00  -0.00
Price        -0.00   1.00


Ahora planteamos nuestro modelo:

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

df_train = df_train[df_train["Customer ID"].isnull() == False]
df_train = df_train[df_train["Quantity"].isnull() == False]

x = df_train[["Customer ID", "Quantity"]]

y = df_train["Price"]


In [25]:
# # Convertir variables categóricas a numéricas utilizando codificado one-hot
# x = pd.get_dummies(x, columns=["Country"])

# Dividir los datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Entrenar un modelo de regresión lineal utilizando el conjunto de entrenamiento
model = LinearRegression()
model.fit(x_train, y_train)

# Hacer predicciones en el conjunto de prueba y evaluar el rendimiento del modelo
y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Error cuadrático medio: {mse:.2f}")

Error cuadrático medio: 13946.75


In [26]:
model = LinearRegression()
model.fit(x_train, y_train)


LinearRegression()

In [27]:
y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Error cuadrático medio: {mse:.2f}")

Error cuadrático medio: 13946.75
